# train 데이터 전처리하기

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/raw/train.csv')

In [4]:
df.head()

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1628180742,0,0.0,353733,0,0.0,127492639,42516,Seizure,3,0,0,0,0,0
1,1628180742,1,6.0,353733,1,6.0,3887563113,42516,Seizure,3,0,0,0,0,0
2,1628180742,2,8.0,353733,2,8.0,1142670488,42516,Seizure,3,0,0,0,0,0
3,1628180742,3,18.0,353733,3,18.0,2718991173,42516,Seizure,3,0,0,0,0,0
4,1628180742,4,24.0,353733,4,24.0,3080632009,42516,Seizure,3,0,0,0,0,0


In [3]:
TARGETS = df.columns[-6:]
print(TARGETS)

Index(['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote',
       'other_vote'],
      dtype='object')


In [43]:
train = (
    df
    .groupby(
        ['eeg_id', 'expert_consensus']
        )[
            [
                'eeg_label_offset_seconds', 
                'spectrogram_id',
                'label_id',
                'spectrogram_label_offset_seconds'
            ]
         ]
    .agg(
        {
            'eeg_label_offset_seconds': 'min', 
            'spectrogram_id': 'first',
            'label_id': 'first',
            'spectrogram_label_offset_seconds': 'min'
        })
)
train.columns = ['eeg_min', 'spec_id', 'label_id', 'spec_min']
train.head()

,,eeg_min,spec_id,label_id,spec_min
eeg_id,expert_consensus,,,,
568657,Other,0.0,789577333,1825637311,0.0
582999,LPD,0.0,1552638400,1722186807,0.0
642382,Other,0.0,14960202,3254468733,1008.0
751790,GPD,0.0,618728447,2898467035,908.0
778705,Other,0.0,52296320,3255875127,0.0


In [44]:
tmp = (
    df
    .groupby(
        ['eeg_id', 'expert_consensus']
        )[['eeg_label_offset_seconds','spectrogram_label_offset_seconds']]
    .agg(
        {
            'eeg_label_offset_seconds': 'max', 
            'spectrogram_label_offset_seconds':'max'
        })
)
train[['eeg_max', 'spec_max']] = tmp
train.head()

,,eeg_min,spec_id,label_id,spec_min,eeg_max,spec_max
eeg_id,expert_consensus,,,,,,
568657,Other,0.0,789577333,1825637311,0.0,16.0,16.0
582999,LPD,0.0,1552638400,1722186807,0.0,38.0,38.0
642382,Other,0.0,14960202,3254468733,1008.0,24.0,1032.0
751790,GPD,0.0,618728447,2898467035,908.0,0.0,908.0
778705,Other,0.0,52296320,3255875127,0.0,0.0,0.0


In [45]:
tmp = (
    df
    .groupby(['eeg_id', 'expert_consensus'])[['patient_id']]
    .agg('first')
)
train['patient_id'] = tmp
train.head()

,,eeg_min,spec_id,label_id,spec_min,eeg_max,spec_max,patient_id
eeg_id,expert_consensus,,,,,,,
568657,Other,0.0,789577333,1825637311,0.0,16.0,16.0,20654
582999,LPD,0.0,1552638400,1722186807,0.0,38.0,38.0,20230
642382,Other,0.0,14960202,3254468733,1008.0,24.0,1032.0,5955
751790,GPD,0.0,618728447,2898467035,908.0,0.0,908.0,38549
778705,Other,0.0,52296320,3255875127,0.0,0.0,0.0,40955


In [46]:
tmp = df.groupby(['eeg_id', 'expert_consensus'])[TARGETS].agg('sum')
for t in TARGETS:
    train[t] = tmp[t].values
    
tmp.head()

,,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
eeg_id,expert_consensus,,,,,,
568657,Other,0,0,12,0,8,28
582999,LPD,0,132,0,11,0,11
642382,Other,0,0,0,0,0,2
751790,GPD,0,0,1,0,0,0
778705,Other,0,0,0,0,0,2


In [47]:
y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

train.head()

,,eeg_min,spec_id,label_id,spec_min,eeg_max,spec_max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
eeg_id,expert_consensus,,,,,,,,,,,,,
568657,Other,0.0,789577333,1825637311,0.0,16.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333
582999,LPD,0.0,1552638400,1722186807,0.0,38.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429
642382,Other,0.0,14960202,3254468733,1008.0,24.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
751790,GPD,0.0,618728447,2898467035,908.0,0.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000
778705,Other,0.0,52296320,3255875127,0.0,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000


In [48]:
train.reset_index(inplace=True)
train.rename(columns={'expert_consensus': 'target'}, inplace=True)
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()

Train non-overlapp eeg_id shape: (18013, 15)


,eeg_id,target,eeg_min,spec_id,label_id,spec_min,eeg_max,spec_max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,568657,Other,0.0,789577333,1825637311,0.0,16.0,16.0,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,582999,LPD,0.0,1552638400,1722186807,0.0,38.0,38.0,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,642382,Other,0.0,14960202,3254468733,1008.0,24.0,1032.0,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,751790,GPD,0.0,618728447,2898467035,908.0,0.0,908.0,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,778705,Other,0.0,52296320,3255875127,0.0,0.0,0.0,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000


In [78]:
train.tail()

,target,eeg_id,eeg_min,eeg_max,spec_id,spec_min,spec_max,label_id,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
18008,GRDA,4293354003,0.0,0.0,1188113564,0.0,0.0,447244163,16610,0.0,0.0,0.0,0.0,0.500000,0.500000
18009,GRDA,4293843368,0.0,0.0,1549502620,0.0,0.0,1618953053,15065,0.0,0.0,0.0,0.0,0.500000,0.500000
18010,Other,4294455489,0.0,0.0,2105480289,0.0,0.0,469526364,56,0.0,0.0,0.0,0.0,0.000000,1.000000
18011,Other,4294858825,0.0,12.0,657299228,0.0,12.0,561576493,4312,0.0,0.0,0.0,0.0,0.066667,0.933333
18012,Other,4294958358,0.0,0.0,260520016,2508.0,2508.0,2788887007,25986,0.0,0.0,0.0,0.0,0.000000,1.000000


In [49]:
train = train[
            ['target', 'eeg_id', 'eeg_min', 'eeg_max',
             'spec_id', 'spec_min', 'spec_max', 'label_id', 'patient_id', 
             'seizure_vote', 'lpd_vote', 'gpd_vote',
             'lrda_vote', 'grda_vote', 'other_vote']
             ]
train.head()

,target,eeg_id,eeg_min,eeg_max,spec_id,spec_min,spec_max,label_id,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,Other,568657,0.0,16.0,789577333,0.0,16.0,1825637311,20654,0.0,0.000000,0.25,0.000000,0.166667,0.583333
1,LPD,582999,0.0,38.0,1552638400,0.0,38.0,1722186807,20230,0.0,0.857143,0.00,0.071429,0.000000,0.071429
2,Other,642382,0.0,24.0,14960202,1008.0,1032.0,3254468733,5955,0.0,0.000000,0.00,0.000000,0.000000,1.000000
3,GPD,751790,0.0,0.0,618728447,908.0,908.0,2898467035,38549,0.0,0.000000,1.00,0.000000,0.000000,0.000000
4,Other,778705,0.0,0.0,52296320,0.0,0.0,3255875127,40955,0.0,0.000000,0.00,0.000000,0.000000,1.000000


eeg_id 당 병명이 여러 개 나온 경우의 index 추출

In [50]:
condition = train['eeg_id'].value_counts()
idx_condition = condition[condition > 1].index
df_multi = train[train['eeg_id'].isin(idx_condition)].copy()
df_multi.sort_values(by=['eeg_id', 'eeg_min'], inplace=True)
df_multi

,target,eeg_id,eeg_min,eeg_max,spec_id,spec_min,spec_max,label_id,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
81,Other,21379701,0.0,158.0,1841034439,2204.0,2362.0,3556945898,32481,0.214286,0.000000,0.0,0.214286,0.000000,0.571429
82,Seizure,21379701,30.0,48.0,1841034439,2234.0,2252.0,1565893564,32481,0.600000,0.000000,0.0,0.100000,0.000000,0.300000
80,LPD,21379701,62.0,110.0,1841034439,2266.0,2314.0,1468675763,32481,0.000000,0.750000,0.0,0.250000,0.000000,0.000000
85,Other,21746311,0.0,0.0,1343094925,0.0,0.0,3249053595,6489,0.400000,0.000000,0.0,0.000000,0.000000,0.600000
86,Seizure,21746311,2.0,54.0,1343094925,2.0,54.0,2917720091,6489,0.720430,0.000000,0.0,0.000000,0.000000,0.279570
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17969,Seizure,4283246598,4.0,6.0,329782959,4.0,6.0,4220128961,27986,1.000000,0.000000,0.0,0.000000,0.000000,0.000000
17980,Other,4284659653,0.0,46.0,1841034439,2696.0,2742.0,2080594355,32481,0.176471,0.000000,0.0,0.176471,0.039216,0.607843
17981,Seizure,4284659653,54.0,58.0,1841034439,2750.0,2754.0,4284231588,32481,0.333333,0.333333,0.0,0.000000,0.000000,0.333333
17978,LPD,4284659653,70.0,138.0,1841034439,2766.0,2834.0,2653269667,32481,0.000000,0.909091,0.0,0.000000,0.000000,0.090909


문제가 되는 경우 예: ['eeg_id'] == 21379701  
이 경우 other가 마지막에 한 번 더 나와서 eeg_max와 spec_max가 전체를 덮는다

In [51]:
df[df['eeg_id'] == 21379701]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
90269,21379701,0,0.0,1841034439,158,2204.0,3556945898,32481,Other,1,0,0,1,0,2
90270,21379701,1,10.0,1841034439,159,2214.0,3225189518,32481,Other,1,0,0,1,0,2
90271,21379701,2,12.0,1841034439,160,2216.0,3059518038,32481,Other,1,0,0,1,0,2
90272,21379701,3,30.0,1841034439,161,2234.0,1565893564,32481,Seizure,2,0,0,1,0,1
90273,21379701,4,44.0,1841034439,162,2248.0,2941979767,32481,Seizure,2,0,0,0,0,1
90274,21379701,5,48.0,1841034439,163,2252.0,4290242864,32481,Seizure,2,0,0,0,0,1
90275,21379701,6,62.0,1841034439,164,2266.0,1468675763,32481,LPD,0,3,0,1,0,0
90276,21379701,7,76.0,1841034439,165,2280.0,4133046008,32481,LPD,0,3,0,1,0,0
90277,21379701,8,78.0,1841034439,166,2282.0,330235701,32481,LPD,0,3,0,1,0,0
90278,21379701,9,80.0,1841034439,167,2284.0,2090915320,32481,LPD,0,3,0,1,0,0


'shift'칼럼을 만들어서 다음 행과 비교하기

In [52]:
df_multi['shift'] = (
    df_multi
    .groupby('eeg_id')['eeg_min']
    .transform('shift', periods= -1, fill_value=np.inf)
)

In [54]:
df_multi[['target', 'eeg_id', 'eeg_min', 'eeg_max', 'shift', 
          'spec_id', 'spec_min', 'spec_max', 'label_id']].head()

,target,eeg_id,eeg_min,eeg_max,shift,spec_id,spec_min,spec_max,label_id
81,Other,21379701,0.0,158.0,30.0,1841034439,2204.0,2362.0,3556945898
82,Seizure,21379701,30.0,48.0,62.0,1841034439,2234.0,2252.0,1565893564
80,LPD,21379701,62.0,110.0,inf,1841034439,2266.0,2314.0,1468675763
85,Other,21746311,0.0,0.0,2.0,1343094925,0.0,0.0,3249053595
86,Seizure,21746311,2.0,54.0,inf,1343094925,2.0,54.0,2917720091


아래는 의사의 판단이 하나의 eeg 내에서 바뀐 경우 찾기

In [67]:
df_view = df_multi[['target', 'eeg_id', 'eeg_min', 'eeg_max', 'shift', 
                    'spec_id', 'spec_min', 'spec_max', 'label_id']]

df_view[df_view['eeg_max'] > df_view['shift']]

,target,eeg_id,eeg_min,eeg_max,shift,spec_id,spec_min,spec_max,label_id
81,Other,21379701,0.0,158.0,30.0,1841034439,2204.0,2362.0,3556945898
129,Other,32067830,0.0,80.0,38.0,2060670605,9422.0,9502.0,2148393111
187,LPD,46287152,0.0,98.0,46.0,1516469502,1232.0,1330.0,3673462713
212,LRDA,51565620,0.0,54.0,18.0,2077600099,2670.0,2724.0,2916140792
235,Other,56878715,0.0,40.0,6.0,1026469553,0.0,40.0,3375752082
...,...,...,...,...,...,...,...,...,...
17796,GPD,4239311496,0.0,290.0,86.0,58683771,2490.0,2780.0,223336409
17836,Other,4247287858,0.0,64.0,20.0,764146759,7652.0,7716.0,3890605516
17835,LRDA,4247287858,94.0,222.0,120.0,764146759,7746.0,7874.0,2315515890
17849,Seizure,4249475128,0.0,256.0,28.0,617579510,0.0,256.0,3097261777


In [79]:
idx_shift = df_view.loc[df_view['eeg_max'] > df_view['shift']].index + 1
idx_shift

Index([   82,   130,   188,   213,   236,   295,   328,   553,   697,   699,
       ...
       17469, 17525, 17550, 17759, 17784, 17797, 17837, 17836, 17850, 17868],
      dtype='int64', length=352)

In [80]:
array_shift = train.loc[idx_shift, 'label_id'].values

In [81]:
df[df['label_id'].isin(array_shift)]

,eeg_id,eeg_sub_id,eeg_label_offset_seconds,spectrogram_id,spectrogram_sub_id,spectrogram_label_offset_seconds,label_id,patient_id,expert_consensus,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
967,628369060,11,76.0,13143748,13,270.0,2050707482,34998,GRDA,0,1,2,1,7,4
990,3088095149,0,0.0,13143748,36,1554.0,1689400488,34998,Other,0,0,0,0,0,2
991,3088095149,1,6.0,13143748,37,1560.0,2169014734,34998,GRDA,0,0,4,0,5,4
1015,3088095149,25,128.0,13143748,61,1682.0,2166878269,34998,Seizure,4,4,2,1,0,2
1625,3204888106,0,0.0,18804669,0,0.0,2589579883,19195,Seizure,3,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105836,3242070203,0,0.0,2122808385,0,0.0,88412723,63251,LRDA,0,0,0,7,1,7
105957,2620906282,0,0.0,2126501728,0,0.0,2956144977,4863,Other,1,5,0,1,1,6
106191,138851420,0,0.0,2131988222,0,0.0,833990299,7797,LRDA,0,0,0,3,0,0
106419,1800361611,0,0.0,2138457516,0,0.0,452006616,58475,Other,1,4,1,0,4,6


In [82]:
idx_new = df[df['label_id'].isin(array_shift)].index -1
df.loc[idx_new, ['eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']]

,eeg_label_offset_seconds,spectrogram_label_offset_seconds
966,64.0,258.0
989,10.0,1234.0
990,0.0,1554.0
1014,118.0,1672.0
1624,0.0,370.0
...,...,...
105835,0.0,1042.0
105956,0.0,0.0
106190,2.0,2.0
106418,0.0,0.0


In [83]:
# 21379701
condition = df.index.isin(idx_new) & (df['eeg_id'] == 21379701)
df.loc[condition, ['eeg_label_offset_seconds', 'spectrogram_label_offset_seconds']]

,eeg_label_offset_seconds,spectrogram_label_offset_seconds
90271,12.0,2216.0
90286,158.0,2362.0


In [61]:
df_view[df_view['eeg_id'] == 1460778765]

,target,eeg_id,eeg_min,eeg_max,shift,spec_id,spec_min,spec_max,label_id
6188,LPD,1460778765,0.0,2070.0,54.0,747230293,2553.0,4623.0,1709929250
6191,Seizure,1460778765,54.0,2088.0,76.0,747230293,2607.0,4641.0,2535836945
6189,LRDA,1460778765,76.0,1668.0,844.0,747230293,2629.0,4221.0,3707315300
6190,Other,1460778765,844.0,1718.0,1228.0,747230293,3397.0,4271.0,2621199549
6187,GRDA,1460778765,1228.0,1228.0,inf,747230293,3781.0,3781.0,1533559374


In [18]:
idx_a = df_t_d3[df_t_d3['max'] > df_t_d3['shift']].index
idx_a

Index([   81,   129,   187,   212,   235,   294,   327,   552,   696,   698,
       ...
       17468, 17524, 17549, 17758, 17783, 17796, 17836, 17835, 17849, 17867],
      dtype='int64', length=352)

In [19]:
train.loc[idx_a, :]

,eeg_id,target,spec_id,min,max,patient_id,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
81,21379701,Other,1841034439,2204.0,2362.0,32481,0.214286,0.000000,0.000000,0.214286,0.000000,0.571429
129,32067830,Other,2060670605,9422.0,9502.0,55705,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
187,46287152,LPD,1516469502,1232.0,1330.0,56450,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
212,51565620,LRDA,2077600099,2670.0,2724.0,41114,0.000000,0.000000,0.000000,0.500000,0.000000,0.500000
235,56878715,Other,1026469553,0.0,40.0,48272,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
17796,4239311496,GPD,58683771,2490.0,2780.0,3838,0.000000,0.123779,0.302932,0.185668,0.185668,0.201954
17836,4247287858,Other,764146759,7652.0,7716.0,30631,0.148649,0.054054,0.000000,0.094595,0.000000,0.702703
17835,4247287858,LRDA,764146759,7746.0,7874.0,30631,0.000000,0.000000,0.000000,0.562500,0.062500,0.375000
17849,4249475128,Seizure,617579510,0.0,256.0,36301,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [41]:
print(df.label_id.nunique())
print(df.shape)

106800
(106800, 15)
